In [84]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [85]:
data = pd.read_csv('agg_flow_data.csv')
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [86]:
label_counts = data['label'].value_counts()
print(label_counts)

label
messaging           39769
regular_browsing     2901
video_stream         1325
file_transfer         275
Name: count, dtype: int64


In [87]:
data.head(5)

,number_of_packets,average_packet_length,number_of_unique_dests,TCP_protocol_count,UDP_protocol_count,Packet_length_variance,Inter_arrival_time_variance,label
0,3,143.00,2,3,0,7959.00,0.27,messaging
1,3,176.67,2,3,0,16124.33,0.73,messaging
2,1,40.00,1,1,0,0.00,0.00,messaging
3,2,198.00,2,2,0,2.00,5.56,messaging
4,4,346.50,2,4,0,24464.33,0.20,messaging


In [88]:
label_counts = data['label'].value_counts()
print(label_counts)

label
messaging           39769
regular_browsing     2901
video_stream         1325
file_transfer         275
Name: count, dtype: int64


In [89]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [107]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [108]:
y_train_encoded = to_categorical(y_train_encoded)

In [109]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [110]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(y_train_encoded.shape[1], activation='softmax')  # Output layer with softmax activation for multi-class classification
])

In [111]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [112]:
model.fit(X_train_scaled, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
12/12 [==============================] - 1s 6ms/step - loss: 1.4347 - accuracy: 0.3116 - val_loss: 1.4135 - val_accuracy: 0.8764
Epoch 2/10
12/12 [==============================] - 0s 2ms/step - loss: 1.2555 - accuracy: 0.8159 - val_loss: 1.2788 - val_accuracy: 0.8764
Epoch 3/10
12/12 [==============================] - 0s 2ms/step - loss: 1.1563 - accuracy: 0.8810 - val_loss: 1.1316 - val_accuracy: 0.8652
Epoch 4/10
12/12 [==============================] - 0s 2ms/step - loss: 0.9985 - accuracy: 0.8952 - val_loss: 0.9417 - val_accuracy: 0.8652
Epoch 5/10
12/12 [==============================] - 0s 2ms/step - loss: 0.8080 - accuracy: 0.9037 - val_loss: 0.7714 - val_accuracy: 0.8764
Epoch 6/10
12/12 [==============================] - 0s 2ms/step - loss: 0.7321 - accuracy: 0.8980 - val_loss: 0.5984 - val_accuracy: 0.8989
Epoch 7/10
12/12 [==============================] - 0s 2ms/step - loss: 0.5901 - accuracy: 0.8952 - val_loss: 0.4684 - val_accuracy: 0.9101
Epoch 8/10
12/12 [==

In [105]:
loss, accuracy = model.evaluate(X_test_scaled, to_categorical(label_encoder.transform(y_test)))
print(f'Test Accuracy: {accuracy}')

1370/1370 [==============================] - 1s 361us/step - loss: 0.3247 - accuracy: 0.9356
Test Accuracy: 0.9355891346931458
